In [83]:
import numpy as np
import pandas as pd

In [84]:
data_train=pd.read_csv("ml_challenges/stockprices/x_train.csv")
data_y=pd.read_csv("ml_challenges/stockprices/y_train.csv")
test_data=pd.read_csv("ml_challenges/stockprices/x_test.csv")
data_train=data_train.values
data_y=data_y.values
test_data=test_data.values

In [85]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
x_train = sc.fit_transform(data_train)
y_train = sc.transform(data_y)


In [86]:
x=[]
y=[]
for i in range(40,1257):
    x.append(x_train[i-40:i, 0])
    y.append(y_train[i,0])

In [87]:
train_x,train_y=np.array(x),np.array(y)

In [88]:
train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 1))

In [106]:
from keras.models import Sequential
from keras.layers import *
model=Sequential()
model.add(LSTM(32,input_shape=(None,1),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(32,return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['acc'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, None, 32)          4352      
_________________________________________________________________
dropout_5 (Dropout)          (None, None, 32)          0         
_________________________________________________________________
lstm_10 (LSTM)               (None, None, 32)          8320      
_________________________________________________________________
dropout_6 (Dropout)          (None, None, 32)          0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dropout_7 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                

In [107]:
from keras.callbacks import ModelCheckpoint
modelcheckpoint = ModelCheckpoint("best_model.h5",save_best_only=True)
hist = model.fit(train_x,train_y, epochs = 50, batch_size = 32, validation_split=0.2,callbacks=[modelcheckpoint])

Train on 973 samples, validate on 244 samples
Epoch 1/50
973/973 [==============================] - 5s 5ms/step - loss: 0.0496 - acc: 0.0010 - val_loss: 0.0101 - val_acc: 0.0041
Epoch 2/50
973/973 [==============================] - 3s 3ms/step - loss: 0.0151 - acc: 0.0010 - val_loss: 0.0021 - val_acc: 0.0041
Epoch 3/50
973/973 [==============================] - 3s 3ms/step - loss: 0.0126 - acc: 0.0010 - val_loss: 0.0081 - val_acc: 0.0041
Epoch 4/50
973/973 [==============================] - 3s 3ms/step - loss: 0.0117 - acc: 0.0010 - val_loss: 0.0019 - val_acc: 0.0041
Epoch 5/50
973/973 [==============================] - 3s 3ms/step - loss: 0.0103 - acc: 0.0010 - val_loss: 0.0018 - val_acc: 0.0041
Epoch 6/50
973/973 [==============================] - 3s 3ms/step - loss: 0.0084 - acc: 0.0010 - val_loss: 0.0042 - val_acc: 0.0041
Epoch 7/50
973/973 [==============================] - 3s 3ms/step - loss: 0.0082 - acc: 0.0010 - val_loss: 0.0036 - val_acc: 0.0041
Epoch 8/50
973/973 [==========

In [108]:
model.load_weights("best_model.h5")

In [109]:
data_train2=pd.read_csv("ml_challenges/stockprices/x_train.csv")
data_test2=pd.read_csv("ml_challenges/stockprices/x_test.csv")
data = pd.concat((data_train2,data_test2), axis=0)

In [110]:
inputs = data[len(data) - 19 - 40:].values
inputs = inputs.reshape(-1,1)
inputs  = sc.transform(inputs)

In [111]:
x_test=[]
for i in range(40,inputs.shape[0]):
    x_test.append(inputs[i-40:i,0])
x_test = np.array(x_test)

In [112]:
x_test=x_test.reshape((x_test.shape[0],x_test.shape[1],1))
pred=model.predict(x_test)
pred=sc.inverse_transform(pred)

In [113]:
pred

array([[792.8239 ],
       [792.16205],
       [791.3203 ],
       [790.50183],
       [790.1339 ],
       [790.38947],
       [791.21436],
       [792.4372 ],
       [793.8702 ],
       [795.43634],
       [796.93896],
       [798.2423 ],
       [799.36194],
       [800.2457 ],
       [801.2232 ],
       [802.5124 ],
       [804.36316],
       [806.7872 ],
       [809.71387]], dtype=float32)

In [114]:
pred2=model.predict(train_x)
pred2=sc.inverse_transform(pred2)

In [115]:
pred2

array([[305.93445],
       [306.4386 ],
       [307.03265],
       ...,
       [792.3777 ],
       [792.9422 ],
       [793.1282 ]], dtype=float32)

In [116]:
data_y

array([[333.87],
       [330.75],
       [328.77],
       ...,
       [794.23],
       [785.93],
       [782.78]])

In [117]:
for i in range(1217):
    print(pred2[i],data_y[i])

[305.93445] [333.87]
[306.4386] [330.75]
[307.03265] [328.77]
[307.65594] [322.29]
[308.1188] [315.72]
[308.3741] [313.52]
[308.4329] [315.26]
[308.3353] [312.3]
[308.0773] [314.81]
[307.80215] [315.82]
[307.63657] [319.3]
[307.634] [294.4]
[307.81644] [293.23]
[308.28577] [292.74]
[309.0123] [288.27]
[310.04602] [286.17]
[311.35562] [289.08]
[312.84955] [288.92]
[314.4328] [290.91]
[316.05725] [291.66]
[317.7135] [292.11]
[319.33755] [297.42]
[320.83777] [304.27]
[322.09592] [303.56]
[323.07513] [304.53]
[323.69223] [306.1]
[323.92618] [302.93]
[323.81433] [305.77]
[323.42654] [304.86]
[322.8626] [305.32]
[322.39133] [303.27]
[322.09076] [302.68]
[321.71075] [307.79]
[321.08774] [307.24]
[320.1467] [302.83]
[318.96338] [304.68]
[317.56238] [305.04]
[315.90994] [308.73]
[314.15088] [311.63]
[312.4562] [311.68]
[311.0003] [310.83]
[309.84344] [310.08]
[308.98044] [303.27]
[308.33112] [304.45]
[307.8008] [304.61]
[307.40353] [304.81]
[307.06274] [302.37]
[306.7669] [307.77]
[306.57657] [

[601.67883] [557.41]
[603.79663] [558.05]
[605.8483] [558.16]
[607.86957] [558.41]
[609.7771] [556.79]
[611.5052] [556.38]
[612.8174] [557.34]
[613.7415] [567.72]
[614.41705] [571.52]
[614.96735] [569.97]
[615.4693] [567.41]
[615.8428] [571.31]
[616.0242] [573.35]
[615.93384] [575.34]
[615.61646] [576.8]
[614.9056] [578.15]
[613.88324] [579.83]
[612.83484] [581.76]
[611.92596] [579.07]
[611.20294] [574.62]
[610.67487] [561.15]
[610.0324] [560.77]
[608.9947] [558.78]
[607.54175] [574.09]
[605.36707] [591.05]
[602.4648] [588.65]
[599.03894] [575.34]
[595.3908] [573.24]
[592.6391] [577.91]
[590.79315] [586.75]
[589.5254] [588.99]
[587.9282] [593.71]
[585.8368] [592.78]
[583.5494] [597.71]
[581.2835] [599.95]
[578.6023] [604.17]
[575.53687] [602.35]
[572.1052] [601.19]
[568.89105] [602.68]
[566.0252] [607.8]
[563.27704] [609.96]
[560.59686] [612.15]
[557.93506] [609.83]
[555.27576] [609.81]
[552.5128] [601.66]
[549.5366] [605.73]
[546.6556] [609.26]
[544.0239] [610.79]
[541.8484] [611.2]
[

[760.2209] [730.59]
[759.29285] [741.]
[757.51953] [737.8]
[754.73914] [731.15]
[751.3658] [729.49]
[747.91296] [731.84]
[744.406] [741.41]
[740.5963] [742.]
[736.59955] [757.92]
[732.5295] [762.71]
[728.8451] [755.28]
[726.0018] [752.]
[724.17554] [753.41]
[723.04395] [754.93]
[722.37384] [768.95]
[722.37354] [775.96]
[723.2923] [768.99]
[725.3788] [768.49]
[729.2767] [768.73]
[734.4775] [764.8]
[739.2346] [764.23]
[742.1755] [755.85]
[742.45215] [745.71]
[740.51855] [748.73]
[737.0329] [758.08]
[732.355] [760.59]
[727.07214] [762.68]
[721.786] [754.13]
[717.18066] [750.]
[713.63135] [754.85]
[710.9942] [754.21]
[709.36145] [751.35]
[708.5305] [762.99]
[708.0938] [779.98]
[707.95416] [777.6]
[708.20135] [769.5]
[708.74835] [744.06]
[709.6836] [752.]
[710.98926] [747.18]
[712.53687] [738.5]
[714.12427] [733.23]
[715.4094] [718.85]
[716.15314] [728.75]
[716.37006] [734.74]
[716.40173] [721.92]
[716.6886] [706.74]
[717.5891] [709.98]
[719.0559] [706.85]
[721.0535] [719.19]
[723.56165] [7

In [120]:
result = pd.DataFrame(pred, columns=['High'])

In [121]:
result.to_csv("pred_updated.csv", index=False)